In [3]:
# HACK: use project root as the working directory 
from pathlib import Path

while Path.cwd().name != 'language-model-toxicity':
    %cd ..

/home/sam/Desktop/research/language-model-toxicity


## Setup model

In [4]:
import pandas as pd
import torch
from transformers import GPT2Tokenizer

from datasets.affect_dataset import create_affect_vector
from models.affect_lm import AffectGPT2LMHeadModel
from utils.generation import GPT2Generator
from utils.constants import OUTPUT_DIR, PERSPECTIVE_API_ATTRIBUTES_LOWER

In [5]:
model_dir = OUTPUT_DIR / 'affect' / 'affect_10k_strat' / 'checkpoint-7868'
model = AffectGPT2LMHeadModel.from_pretrained(model_dir)

In [6]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

## Inspect largest weights

In [7]:
affect2vocab = model.affect.affect2vocab
values, indices = affect2vocab.weight.sort(dim=0)
top_vocab = reversed(indices[:,3])

In [8]:
vocab = tokenizer.get_vocab()
vocab, _ = zip(*vocab.items())
vocab = pd.Series(vocab)

# Print top weighted vocab
vocab[top_vocab.tolist()][:20]

50256    <|endoftext|>
247                  Ļ
251                  Ŀ
1714              Ġsex
11740            ĠDick
250                  ľ
3206           Ġsexual
8483             Ġporn
11333           Ġballs
11363        Ġsexually
564                ĠâĢ
7262             Ġrape
22938         Ġmasturb
10110            Ġsuck
16360           Ġpenis
35853        Ġgenitals
19311           Ġsperm
840               Ġass
29836         Ġasshole
16110           Ġraped
dtype: object

## Generations

In [9]:
generator = GPT2Generator(model)
device = generator.device

In [24]:
model.affect.beta = 1

In [25]:
prompt = "In this paper "

In [27]:
# Sexually explicit generations
affect_labels = [create_affect_vector(sexually_explicit=1.5) for i in range(20)]
affect_labels = torch.tensor(affect_labels).unsqueeze(dim=1).to(device)

generator([prompt] * len(affect_labels),
          max_len=50,
          affect_labels=affect_labels)

[', we discuss the finding that Adels and others have evidence of P-si to the association between association of methamphetamine with cocaine drug test results and P for cocaine drug test test score, in combination with knowledge about the possible hypothesis of drug involvement. Con',
 ', we show that if we put in some of the slowest, they may even have the biggest impact on the world. Also, while the arithmetical gap is lower for a bowling club, it still has a major effect on your',
 ', the exposure and impact of unwanted air use and drug use on breastfeeding outcomes are examined in a prospective study with a representative sample of 60,000 breast cancer survivors in the United States. Burden of breast cancer and later breast cancer in the first six',
 ', we reported that HIV infection in HIV-positive male sex partners ( is thought to be in the lowest proportions in the sexually transmitted diseases, HIV, HB, and P). The lowest percentage of these sexual partners has been found to h

In [29]:
affect_labels.shape

torch.Size([20, 1, 4])

In [28]:
# Generate with all affect vectors
affect_labels = [create_affect_vector(**{attribute: 1}) for attribute in PERSPECTIVE_API_ATTRIBUTES_LOWER]
affect_labels = torch.tensor(affect_labels).unsqueeze(dim=1).to(device)
out = generator([prompt] * len(affect_labels), max_len=50, affect_labels=affect_labels)

for attribute, gen in zip(PERSPECTIVE_API_ATTRIBUTES_LOWER, out):
    print(
        'GENERATION WITH ' + attribute.upper(), 
        prompt + gen,
        sep='\n', end='\n\n'
    )

TypeError: create_affect_vector() got an unexpected keyword argument 'toxicity'

In [90]:
# Generate without affect vectors (sanity check)
generator.generate_multiple(prompt, max_len=50, num_return_sequences=20)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


[' won his full time gig playing the "Lady and the Tramp" prequel, "Lady and the Tramp." In 2017, he will play the character and spearhead his Broadway show, "The Funny Woman" at the Broadway Winter Showcase',
 ' claims that her workout schedule means he can "slip off the top of any car" and a job may be possible with him on the show.\n\nThe sketch has been deemed one of the most humorous on a "hit show" as he',
 " is retiring from television after 30 years.\n\nHis departure came as the Court of Appeal acknowledged that it was guilty of cronyism after hearing testimony that Stewart approached Her Majesty's Treasury to urge a number of her staff to meet with the Prime Minister",
 ' denied reports that she did not take a train to France last October.\n\nElbowmaker Arnaud Dessee, a 68-year-old Frenchman of French descent, has reportedly confessed to terror attacks in Brussels and Paris, her lawyer told',
 ' is in talks to open his own theater. He now produces comedy with his co-anchor on 